# Density matrix exponentiation
## Overview
This is a function to simulate DME. It follows the protocol laid down by https://arxiv.org/abs/2001.08838

Goals: implement the unitary $U=e^{-i\rho\theta}$ on data qubit $\sigma$ (rotate $\sigma$ by $\theta$) according to instruction given by instruction qubit $\rho$.

DME uses the relation (rotate $\sigma$ by small angle $\delta = \sigma/N$).
$$\begin{align}
Tr_\rho [e^{-iSWAP\delta}\sigma\otimes\rho e^{iSWAP\delta}] &= \sigma - i\delta[\rho,\sigma] +\mathcal{O}(\delta^2)\\
&= e^{-i\rho\delta}\sigma e^{i\rho\delta}+\mathcal{O}(\delta^2)
\end{align}$$

Reference
- Kjaergaard, M., Schwartz, M. E., Greene, A., Samach, G. O., Bengtsson, A., O'Keeffe, M., ... Oliver, W. D. (2020). 
    Programming a quantum computer with quantum instructions. arXiv preprint arXiv:2001.08838. 
    Retrieved from https://arxiv.org/abs/2001.08838 


## Code exlanation
In this code, we set $\theta = \pi, N =k$ with 1 work qubit and $k$ instruction qubits (it is possible to set more instruction qubits here, but they will just be idle). 

The instruction qubits are thrown out after usage (We call it Sequential Instruction Qubit protocol). Each query use, accordingly, k qubits. Each call use 1 query.

Our goal is to rotate qubit 0 from state $\ket{0}$ to state $\ket{1}$ by using an $RX(\pi)$ pulse. We demonstrate this by showing the number of counter that return 1.

### Test with 3 iterations

In [ ]:
from qibo.models.qdp.quantum_dynamic_programming import DensityMatrixExponentiation
import numpy as np
from qibo import gates
k = 3
my_protocol = DensityMatrixExponentiation(theta=np.pi,N=k,num_work_qubits=1,num_instruction_qubits=k,number_muq_per_call=1)
my_protocol.memory_call_circuit(num_instruction_qubits_per_query=k)
print('Circuit for DME, q0 is target qubit, q1,q2 and q3 are instruction qubit')
print(my_protocol.c.draw())
my_protocol.c.add(gates.M(0))
my_dict = my_protocol.c.execute(nshots=1000).frequencies(registers = True)
my_counter = my_dict['register0']
print(f'Count of qubit 0: {my_counter}')


### Test with 20 iterations

In [ ]:
k = 20
my_protocol = DensityMatrixExponentiation(theta=np.pi,N=k,num_work_qubits=1,num_instruction_qubits=k,number_muq_per_call=1)
my_protocol.memory_call_circuit(num_instruction_qubits_per_query=k)
#print('Circuit for DME, q0 is target qubit, q1,q2 and q3 are instruction qubit')
#print(my_protocol.c.draw())
my_protocol.c.add(gates.M(0))
my_dict = my_protocol.c.execute(nshots=1000).frequencies(registers = True)
my_counter = my_dict['register0']
print(f'Count of qubit 0: {my_counter}')

In [ ]:
iterations = 20
count_1 = np.zeros(iterations)
for k in range(1,iterations+1):
    my_protocol = DensityMatrixExponentiation(theta=np.pi,N=k,num_work_qubits=1,num_instruction_qubits=k,number_muq_per_call=1)
    my_protocol.memory_call_circuit(num_instruction_qubits_per_query=k)
    #print('Circuit for DME, q0 is target qubit, q1,q2 and q3 are instruction qubit')
    #print(my_protocol.c.draw())
    my_protocol.c.add(gates.M(0))
    my_dict = my_protocol.c.execute(nshots=1000).frequencies(registers = True)
    my_counter = my_dict['register0']
    # print(f'Count of qubit 0: {my_counter}')
    count_1[k-1] = my_counter['1']
count_1 /= 1000

In [ ]:
import matplotlib.pyplot as plt
plt.plot(count_1)
plt.title('Transition rate using QDP')
plt.xlabel('Iterations')
plt.ylabel(r'Count of $|1\rangle$')